In [1]:
import sys
sys.path.insert(1, './env')
sys.path.insert(1, './agents')

import gym
import pylab
import random
import strategy_imitation, sarsa, ddqn, random_agent, a2c
import aa_gun
import numpy as np
from collections import deque
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
#массовый тест моделей

In [ ]:
#Проверь на зенитке, на cartpole и на mountain car

score_dict_full={'random':[],'ddqn':[],'a2c':[],'sarsa':[]}
border_med = 100
EPISODES=140
score_dict_med={'random':[],'ddqn':[],'a2c':[],'sarsa':[]}#c border_med по... такты. Надо, чтобы проверить быстроту обучения

agent_list=[random_agent.randomAgent,a2c.A2CAgent,ddqn.DoubleDQNAgent,sarsa.SarsaAgent]
for ag_num in range(4):
    if ag_num==0:
        name='random'
    if ag_num==1:
        name='a2c'
    if ag_num==2:
        name='ddqn' 
    if ag_num==3:
        name='sarsa'
    
    for estimation in range(5):
        print('_____',name,estimation)
        #здесь весь код от инициализации модели до выдачи scores. Но без рендера.
        # In case of CartPole-v1, maximum length of episode is 500
        
        env = aa_gun.AA_gun_simple0_env()
        #env = gym.make('MountainCar-v0')
        
        #env=CartPoleEnv9()
        # get size of state and action from environment
        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        #agent = DoubleDQNAgent(state_size, action_size)
        agent = agent_list[ag_num](state_size, action_size)
        agent.train_start=100
        agent.render=True

        scores, episodes = [], []
        reward_lst = []
        s_list=[]
        a_list=[]

        for e in range(EPISODES):
            done = False
            score = 0
            state = env.reset()
            state = np.reshape(state, [1, state_size])

            while not done:
                #if (e in range(2,7)) or (e in range(20,25)) or (e in range(100,103)) or (e in range(200,202)) or (e in range(300,306)) or (e in range(400,406)) or (e in range(500,506)) or (e in range(600,604)):
                #    if agent.render:
                #        env.render()

                # get action for the current state and go one step in environment
                action = agent.get_action(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state, [1, state_size])
                # if an action make the episode end, then gives penalty of -100

                
                # save the sample <s, a, r, s'> to the replay memory
                agent.append_sample(state, action, reward, next_state, done)
                #if next_state[0,11]!=reward:
                #    print('state[13]!=reward',state[0,11],reward)
                #
                s_list.append(state)
                a_list.append(action)
                reward_lst.append(reward)
                #

                # every time step do the training
                agent.train_model()
                score += reward
                state = next_state

                if done:
                    # every episode update the target model to be same with model
                    agent.update_target_model()

                    # every episode, plot the play time
                    scores.append(score)
                    episodes.append(e)
                    pylab.plot(episodes, scores, 'b')
                    #pylab.savefig("./save_graph/aa_gun_dqn.png")
                    print("episode:", e, "  score:", score, "  memory length:",
                          len(agent.memory), "  epsilon:", agent.epsilon)



            # save the model
            #if e % 50 == 0:
            #    agent.model.save_weights("./save_model/aa_gun_dqn.h5")



        #и первые 3000 тактов - это рандом
        #Ходов так 50
        score_dict_full[name].append(np.mean(scores))
        score_dict_med[name].append(np.mean(scores[border_med:]))
        import pickle
        f=open('score_dict.pkl','wb')
        pickle.dump([score_dict_full,score_dict_med],f)
        plt.show()

_____ random 0
hit
episode: 0   score: 1.9375   memory length: 0   epsilon: 1
hit
hit
episode: 1   score: 3.3125   memory length: 0   epsilon: 1
episode: 2   score: 0   memory length: 0   epsilon: 1
episode: 3   score: 0   memory length: 0   epsilon: 1
hit
episode: 4   score: 1.4375   memory length: 0   epsilon: 1
episode: 5   score: 0   memory length: 0   epsilon: 1
episode: 6   score: 0.5625   memory length: 0   epsilon: 1
episode: 7   score: 0.25   memory length: 0   epsilon: 1
episode: 8   score: 0.75   memory length: 0   epsilon: 1
hit
episode: 9   score: 1.5   memory length: 0   epsilon: 1
episode: 10   score: 0.5625   memory length: 0   epsilon: 1
hit
episode: 11   score: 2.125   memory length: 0   epsilon: 1
hit
episode: 12   score: 1.8125   memory length: 0   epsilon: 1
hit
episode: 13   score: 1.8125   memory length: 0   epsilon: 1
hit
episode: 14   score: 1.3125   memory length: 0   epsilon: 1
hit
episode: 15   score: 2.25   memory length: 0   epsilon: 1
hit
hit
episode: 16 